In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [12]:
sm.__version__

'0.13.5'

In [5]:
interval_and_weather_merged = pd.read_csv('./interval_and_weather_merged.csv')

In [25]:
# Data Cleanup including Removing American Aster
interval_and_weather_merged = interval_and_weather_merged[(interval_and_weather_merged['sheet_title'] != '10-14-2022 i11 American Aster') & (interval_and_weather_merged['sheet_title'] != '10-14-2022 i8 American Aster')]

# Interval column is in nanoseconds
interval_cut_off = 1*60*1000*1000000
df_less_than_interval = interval_and_weather_merged[(interval_and_weather_merged['interval'] < interval_cut_off) & (interval_and_weather_merged['same_insect'] == True)]

df_minus_short_interval_same_insect = interval_and_weather_merged[(interval_and_weather_merged['interval'] >= interval_cut_off) | (interval_and_weather_merged['same_insect'] == False)]
df_minus_short_interval_same_insect

,Unnamed: 0,endtime,starttime,interval,departing_insect,arriving_insect,sheet_title,timestamp_pd,timestamp_pd_rounded,same_insect,...,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,temp_group
0,0,0 days 11:41:43,0 days 11:44:13,150000000000,Insecta,Insecta,07-28-2022 i10 Zinnia,2022-07-28 11:41:43,2022-07-28 12:00:00,1,...,100.0,0.0,NaN,270.0,7.2,NaN,1014.0,NaN,3.0,Medium
1,1,0 days 11:44:43,0 days 12:04:13,1170000000000,Insecta,Syrphinae,07-28-2022 i10 Zinnia,2022-07-28 11:44:43,2022-07-28 12:00:00,0,...,100.0,0.0,NaN,270.0,7.2,NaN,1014.0,NaN,3.0,Medium
2,2,0 days 13:43:07,0 days 13:54:07,660000000000,Halictus ligatus/poeyi,Syrphidae,07-28-2022 i4 Black-eyed Susan,2022-07-28 13:43:07,2022-07-28 14:00:00,0,...,79.0,0.0,NaN,271.0,8.3,NaN,1013.0,NaN,3.0,High
3,3,0 days 12:57:16,0 days 12:59:20,124000000000,Augochlorini,Apis mellifera,08-04-2022 i8 Cup Plant,2022-08-04 12:57:16,2022-08-04 13:00:00,1,...,84.0,0.0,NaN,220.0,7.0,NaN,1019.0,NaN,1.0,High
5,5,0 days 13:01:55,0 days 13:03:28,93000000000,Augochlorini,Augochlorini,08-04-2022 i8 Cup Plant,2022-08-04 13:01:55,2022-08-04 13:00:00,1,...,84.0,0.0,NaN,220.0,7.0,NaN,1019.0,NaN,1.0,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,260,0 days 00:48:56,0 days 01:09:20,1224000000000,B. impatiens (F),B. impatiens (F),(pol.) +Bidens ferufillioa 10.11.22,2022-10-11 14:24:27,2022-10-11 14:00:00,1,...,55.0,0.0,NaN,258.0,2.5,NaN,1029.0,NaN,1.0,Low
293,271,0 days 00:00:41,0 days 00:03:07,146000000000,B. impatiens (F),B. impatiens (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:19:29,2022-10-14 13:00:00,1,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN
294,272,0 days 00:03:14,0 days 00:09:39,385000000000,B. impatiens (F),B. impatiens (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:21:59,2022-10-14 13:00:00,1,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN
295,273,0 days 00:09:57,0 days 00:10:14,17000000000,B. impatiens (F),Lasioglossum (Dialictus) sp. (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:28:31,2022-10-14 13:00:00,0,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN


In [35]:
def run_cox_regression_on_all_data(interval_and_weather_merged):
  # JCB: List of ones to represent that all intervals have completed
  status = np.ones(interval_and_weather_merged.shape[0])

  mod = smf.phreg("interval ~ same_insect + temp + wspd + dwpt + coco",
                  interval_and_weather_merged, status=status, ties="efron")
  rslt = mod.fit()
  print(rslt.summary())
  
run_cox_regression_on_all_data(interval_and_weather_merged)
interval_and_weather_merged

                         Results: PHReg
Model:                      PH Reg         Sample size:       246
Dependent variable:         interval       Num. events:       246
Ties:                       Efron                                
-----------------------------------------------------------------
             log HR log HR SE   HR      t    P>|t|  [0.025 0.975]
-----------------------------------------------------------------
same_insect  0.3252    0.1468 1.3843  2.2151 0.0268 1.0382 1.8459
temp         0.0317    0.0272 1.0322  1.1685 0.2426 0.9787 1.0887
wspd         0.0301    0.0158 1.0306  1.9039 0.0569 0.9991 1.0630
dwpt        -0.0913    0.0241 0.9127 -3.7876 0.0002 0.8706 0.9569
coco         0.3863    0.0606 1.4716  6.3754 0.0000 1.3068 1.6572
Confidence intervals are for the hazard ratios


,Unnamed: 0,endtime,starttime,interval,departing_insect,arriving_insect,sheet_title,timestamp_pd,timestamp_pd_rounded,same_insect,...,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,temp_group
0,0,0 days 11:41:43,0 days 11:44:13,150000000000,Insecta,Insecta,07-28-2022 i10 Zinnia,2022-07-28 11:41:43,2022-07-28 12:00:00,1,...,100.0,0.0,NaN,270.0,7.2,NaN,1014.0,NaN,3.0,Medium
1,1,0 days 11:44:43,0 days 12:04:13,1170000000000,Insecta,Syrphinae,07-28-2022 i10 Zinnia,2022-07-28 11:44:43,2022-07-28 12:00:00,0,...,100.0,0.0,NaN,270.0,7.2,NaN,1014.0,NaN,3.0,Medium
2,2,0 days 13:43:07,0 days 13:54:07,660000000000,Halictus ligatus/poeyi,Syrphidae,07-28-2022 i4 Black-eyed Susan,2022-07-28 13:43:07,2022-07-28 14:00:00,0,...,79.0,0.0,NaN,271.0,8.3,NaN,1013.0,NaN,3.0,High
3,3,0 days 12:57:16,0 days 12:59:20,124000000000,Augochlorini,Apis mellifera,08-04-2022 i8 Cup Plant,2022-08-04 12:57:16,2022-08-04 13:00:00,1,...,84.0,0.0,NaN,220.0,7.0,NaN,1019.0,NaN,1.0,High
4,4,0 days 13:00:22,0 days 13:00:53,31000000000,Apis mellifera,Augochlorini,08-04-2022 i8 Cup Plant,2022-08-04 13:00:22,2022-08-04 13:00:00,1,...,84.0,0.0,NaN,220.0,7.0,NaN,1019.0,NaN,1.0,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,271,0 days 00:00:41,0 days 00:03:07,146000000000,B. impatiens (F),B. impatiens (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:19:29,2022-10-14 13:00:00,1,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN
294,272,0 days 00:03:14,0 days 00:09:39,385000000000,B. impatiens (F),B. impatiens (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:21:59,2022-10-14 13:00:00,1,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN
295,273,0 days 00:09:57,0 days 00:10:14,17000000000,B. impatiens (F),Lasioglossum (Dialictus) sp. (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:28:31,2022-10-14 13:00:00,0,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN
296,274,0 days 00:10:36,0 days 00:10:39,3000000000,Lasioglossum (Dialictus) sp. (F),L. (Dialictus) sp. (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:29:06,2022-10-14 13:00:00,1,...,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0,NaN


In [32]:
def run_bin_temp_analysis(interval_and_weather_merged):
    # Find the range of the `temp` column
    temp_min = interval_and_weather_merged['temp'].min()
    temp_max = interval_and_weather_merged['temp'].max()

    print("Photo data temp range")
    print(temp_min)
    print(temp_max)

    # Calculate the range of each group based on the total range of `temp`
    group_range = (temp_max - temp_min) / 3
    print(group_range)
    # Create a new column in the DataFrame to hold the temperature group for each row
    interval_and_weather_merged['temp_group'] = pd.cut(interval_and_weather_merged['temp'], bins=[temp_min, temp_min+group_range, temp_min+2*group_range, temp_max], labels=['Low', 'Medium', 'High'])

    # Print out the DataFrame grouped by temperature group
    for name, group in interval_and_weather_merged.groupby('temp_group'):
        print(f"Group: {name}\n")
        print(f"Number of rows in group: {len(group)}\n")
        status = np.ones(len(group))

        mod = smf.phreg("interval ~ same_insect + temp + wspd + dwpt + coco",
                        group, status=status, ties="efron")
        rslt = mod.fit()
        print(rslt.summary())
# df_minus_short_interval_same_insect

run_bin_temp_analysis(df_minus_short_interval_same_insect)

Photo data temp range
12.0
29.0
5.666666666666667
Group: Low

Number of rows in group: 57

                         Results: PHReg
Model:                     PH Reg          Sample size:        57
Dependent variable:        interval        Num. events:        57
Ties:                      Efron                                 
-----------------------------------------------------------------
             log HR log HR SE   HR      t    P>|t|  [0.025 0.975]
-----------------------------------------------------------------
same_insect  0.1351    0.4824 1.1447  0.2801 0.7794 0.4447 2.9463
temp         0.2290    0.2552 1.2573  0.8972 0.3696 0.7625 2.0733
wspd         0.0860    0.0488 1.0898  1.7630 0.0779 0.9904 1.1991
dwpt        -0.2010    0.0725 0.8179 -2.7719 0.0056 0.7095 0.9428
coco         1.0960    0.4956 2.9923  2.2117 0.0270 1.1329 7.9037
Confidence intervals are for the hazard ratios
Group: Medium

Number of rows in group: 112



/var/folders/vt/546593w53sx4zw12xsp5qhx00000gp/T/ipykernel_52508/2888548076.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interval_and_weather_merged['temp_group'] = pd.cut(interval_and_weather_merged['temp'], bins=[temp_min, temp_min+group_range, temp_min+2*group_range, temp_max], labels=['Low', 'Medium', 'High'])


                         Results: PHReg
Model:                      PH Reg         Sample size:       112
Dependent variable:         interval       Num. events:       112
Ties:                       Efron                                
-----------------------------------------------------------------
             log HR log HR SE   HR      t    P>|t|  [0.025 0.975]
-----------------------------------------------------------------
same_insect  0.1042    0.2028 1.1098  0.5140 0.6073 0.7459 1.6514
temp         0.0905    0.0934 1.0947  0.9685 0.3328 0.9115 1.3147
wspd         0.0453    0.0282 1.0463  1.6068 0.1081 0.9901 1.1057
dwpt        -0.0756    0.0363 0.9271 -2.0867 0.0369 0.8635 0.9954
coco         0.3334    0.1125 1.3957  2.9625 0.0031 1.1194 1.7402
Confidence intervals are for the hazard ratios
Group: High

Number of rows in group: 38

                         Results: PHReg
Model:                     PH Reg          Sample size:        38
Dependent variable:        interval    